In [66]:
import json
from os import listdir
from os.path import isfile, join
from pathlib import Path
import requests
import urllib.request

from tqdm import tqdm

In [67]:
levelFolder = Path("levels/")
levelsDownloaded = set()

In [68]:
onlyfiles = [f for f in listdir(levelFolder) if isfile(join(levelFolder, f))]
levelsDownloaded = set(map(lambda x: x[0 : len(x) - 4], onlyfiles))

In [69]:
def urlBuilder(url, chunk, current):
    return url.format(chunk,current)

In [63]:
def downloadLevelFile(idLevel):
    urllib.request.urlretrieve("https://api.smmdb.net/courses2/download/{idl}".format(idl = idLevel), join(levelFolder, idLevel + '.tar'))

In [64]:
chunkDownload = 100
currentDownloadPosition = 0
baseUrl = "https://api.smmdb.net/courses2?limit={:d}&skip={:d}"
urlToCall = urlBuilder(baseUrl, chunkDownload, currentDownloadPosition)

In [65]:
finish = False
while not finish:
    r = requests.get(urlToCall)
    levelsJson = json.loads(r.content)
    levelsId = set(map(lambda x: x['id'], levelsJson))
    levelsId -= levelsDownloaded #filtro di quelli già scaricati
    [downloadLevelFile(idLevel) for idLevel in levelsId]
    if len(levelsJson) < chunkDownload:
        finish = True
    else:
        currentDownloadPosition += chunkDownload
        urlToCall = urlBuilder(baseUrl, chunkDownload, currentDownloadPosition)
        